# A- Generate daily plan

## Daily plan JSON structure

In [1]:
plan = {
    "date": "",
    
    "start_location": "",
    "places_to_visit": [
        {
            "name": "",
            "location": "",
            "validated": False
        },
        {
            "name": "",
            "location": "",
            "validated": False
        }
    ],
    "sport_exercises": [
        {
            "exercise": "",
            "count_goal": 0,
            "count_done": 0,
        },
        {
            "exercise": "",
            "count_goal": 0,
            "count_done": 0,
        }
    ],
    "gained_coins": 0
}

## Find places to visit in the near by
## Google Places API - Nearby Search 

In [26]:
import random
import requests
import googlemaps # Github: https://github.com/googlemaps/google-maps-services-python/tree/master
import APIs_keys as keys #API_keys.py has been added to .gitignore

In [3]:
#init python-google-maps-api client
try:
    del gmaps
except:
    pass
gmaps = googlemaps.Client(key=keys.GOOGLE_PLACES_API_KEY)

In [51]:
# for tinkering, use Aschaffenburg, DE long and lat. Later we get them from the web browser geolocation.
start_location = {
    "lat": 49.9807,
    "long": 9.1356,
}

# another way to get geometry is to use gmaps Geocode API
"""
geocode_result = gmaps.geocode('Aschaffenburg Hbf, Aschaffenburg, DE')
geocode_result[0]["geometry"]["location"]
res: 
{'lat': 49.98056, 'lng': 9.143699999999999}
"""

# distance from search center in meter (int)
search_radius = 30000 #30KM

# gmaps place types. Full supported values: https://developers.google.com/maps/documentation/places/web-service/supported_types
motivated_work_plc_types = [
    "library", "university"
]
lazy_work_plc_types = [
    "bakery", "cafe", "restaurant", "shopping_mall"
]
fun_plc_types = [
    "amusement_park", "aquarium", "art_gallery", "jewelry_store", "movie_theater", "museum",
    "park", "tourist_attraction"
]

# Google Places results filtering criteria
MAX_N_PLCS = 2
MIN_RATING = 3.5
INCLUDE_MAPS_AND_PHOTOS = True

In [52]:
def filter_places(gmaps_places_res, max_n_plcs, min_rating=0, include_maps_and_photos=True):
    lst_fun_plcs = []
    for place in gmaps_places_res["results"]:
        name = place["name"]
        if "rating" in place:
            rating = place["rating"]
        else:
            rating = "NA"
        if rating == "NA" or rating < min_rating:
            continue # exclude this place
        try:
            maps_and_photos = place["photos"][0]["html_attributions"][0].replace('<a href="', '').split('">')[0]
        except:
            maps_and_photos = "NA"
            if include_maps_and_photos:
                continue # exclude this place
                
        lst_fun_plcs.append(
            {
                "name": name,
                "rating": rating,
                "maps_and_photos": maps_and_photos
            }
        )

    # filter results to max_n_places
    return random.sample(lst_fun_plcs, max_n_plcs)

In [53]:
# Generate Work Places
feel_motivated = True
work_plcs = gmaps.places_nearby(
    location = (start_location["lat"], start_location["long"]),
    radius = search_radius,
    type = motivated_work_plc_types if feel_motivated else lazy_work_plc_types,
    language = 
    "en")

print("Found {0} work places.".format(len(work_plcs["results"])))

# pick random 2 matching the criteria
rnd_work_plcs = filter_places(work_plcs, max_n_plcs = MAX_N_PLCS, min_rating = MIN_RATING, include_maps_and_photos=INCLUDE_MAPS_AND_PHOTOS)
for plc in rnd_work_plcs:
    print(plc)

Found 20 work places.
{'name': 'Stadtbücherei Babenhausen', 'rating': 4.9, 'maps_and_photos': 'https://maps.google.com/maps/contrib/117290890366072471342'}
{'name': 'Stadtbücherei Obernburg', 'rating': 4.9, 'maps_and_photos': 'https://maps.google.com/maps/contrib/102901960301702614600'}


In [54]:
# Generate Fun Places
fun_plcs = gmaps.places_nearby(
    location = (start_location["lat"], start_location["long"]),
    radius = search_radius,
    type = fun_plc_types,
    language = "en")

print("Found {0} fun places.".format(len(fun_plcs["results"])))

# pick random 2 matching the criteria
rnd_fun_plcs = filter_places(fun_plcs, max_n_plcs = MAX_N_PLCS, min_rating = MIN_RATING, include_maps_and_photos=INCLUDE_MAPS_AND_PHOTOS)
for plc in rnd_fun_plcs:
    print(plc)

Found 16 fun places.
{'name': 'Licht- und Luftbad Hanau e.V.', 'rating': 4.5, 'maps_and_photos': 'https://maps.google.com/maps/contrib/109345864326629900080'}
{'name': "Jump'n Fly Trampolinpark Aschaffenburg", 'rating': 4.4, 'maps_and_photos': 'https://maps.google.com/maps/contrib/109296309447529464897'}
